<a href="https://colab.research.google.com/github/cleycianefarias/Rede-Neural-MLP/blob/main/Projeto_de_IC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [32]:
from __future__ import print_function
from builtins import range
import pandas as pd

In [ ]:

"""
SECTION 1: carregar e treinar os dados da base
os conjuntos de dados foram separados em dois para executar o treinamento dos dados
country.csv = conjuntos de dados para fins de treinamento, 70% dos dados originais
train.csv  = conjuntos de dados para fins de teste, 30% dos dados originais
"""
#Carregamento dos dados
datatrain = pd.read_csv('/content/country.csv')

#alterar o valor da String para numérico
datatrain.loc[datatrain['status']=='Closed', 'status']=0
datatrain.loc[datatrain['status']=='Merged', 'status']=1
datatrain = datatrain.apply(pd.to_numeric)

#alterando o dataframe para o array 
datatrain_array = datatrain.values

#dividir x e y (recurso e destino)
xtrain = datatrain_array[:,:18]
ytrain = datatrain_array[:,18]

print("Deu certo")


In [ ]:
"""
SECTION 2 : Construindo e treinando o modelo de adaptação da rede mlp
Multilayer perceptron model (MLP).
input layer : 18 neuron, representa as principais caracteristicas da base
hidden layer : 1 neuron, ativação usando ReLU
output layer : 3 neuron, representa a classe do status final de aceitação/rejeição, Softmax Layer
optimizer = descida gradiente estocástica sem tamanho de lote
loss function = entropia cruzada categórica
learning rate = default from keras.optimizer.SGD, 0.01
epoch = 500
"""
import tensorflow as tf
from tensorflow.keras.layers import Dense, Activation
from tensorflow.keras.utils import to_categorical

#mudar o formato do alvo
ytrain = to_categorical(ytrain) 

#Construindo o modelo
model = tf.keras.Sequential()
model.add(Dense(1, input_shape=(18,)))
model.add(Activation("relu"))
model.add(Dense(2))
model.add(Activation("softmax"))

#escolhendo o otimizador e a função de perda
model.compile(loss='categorical_crossentropy', optimizer='sgd', metrics=['accuracy'])

#treinando o modelo
model.fit(xtrain, ytrain, epochs=500, batch_size=32)
print("parece que funcionou")

In [ ]:

"""
SECTION 3 : Testando o modelo
"""
#carregando o modelo
datatest = pd.read_csv('/content/train.csv')

datatest.loc[datatest['status']=='Closed', 'status']=0
datatest.loc[datatest['status']=='Merged', 'status']=1
datatest = datatest.apply(pd.to_numeric)

#mudando o dataframe para array
datatest_array = datatest.values

#dividir x e y (recurso e destino)
xtest = datatest_array[:,:18]
ytest = datatest_array[:,18]

#pegando a predição
classes = model.predict_classes(xtest, batch_size=32)

#pegando o valor da aurácia de treinamento
import numpy as np
accuration = np.sum(classes == ytest)/len(ytest) * 100

print("Test Accuration : " + str(accuration) + '%')
print("Prediction :")
print(classes)
print("Target :")
print(np.asarray(ytest,dtype="int32"))